In [3]:
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
for gpu in tf.config.experimental.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)


In [4]:
def nin_block(num_channels, kernel_size, strides, padding):
    blk = keras.Sequential([
        layers.Conv2D(num_channels, kernel_size=kernel_size, 
                      strides=strides, padding=padding, activation='relu'), 
        layers.Conv2D(num_channels, kernel_size=1, activation='relu'), 
        layers.Conv2D(num_channels, kernel_size=1, activation='relu'), 
    ])
    return blk


In [12]:
net = keras.Sequential([
    nin_block(96, kernel_size=11, strides=4, padding='valid'), 
    layers.MaxPool2D(pool_size=3, strides=2), 
    
    nin_block(256, kernel_size=5, strides=1, padding='same'), 
    layers.MaxPool2D(pool_size=3, strides=2), 
    
    nin_block(384, kernel_size=3, strides=1, padding='same'), 
    layers.MaxPool2D(pool_size=3, strides=2), 
    
    layers.Dropout(0.5), 
    
    # 使用全局平均池化层对每个通道中所有元素求平均并直接用于分类。
    # 这里的全局平均池化层即窗口形状等于输入空间维形状的平均池化层。
    # NiN的这个设计的好处是可以显著减小模型参数尺寸，从而缓解过拟合。
    # 然而，该设计有时会造成获得有效模型的训练时间的增加。
    nin_block(10, kernel_size=3, strides=1, padding='same'), 
    layers.GlobalAveragePooling2D(), 
    
    layers.Flatten(),
])

In [16]:
input_shape = (1, 224, 224, 1)
X = tf.random.uniform(input_shape)
for blk in net.layers:
    X = blk(X)
    print(blk.name, '\t', X.shape)

net.build(input_shape=input_shape)
print(net.summary())

sequential_5 	 (1, 54, 54, 96)
max_pooling2d_3 	 (1, 26, 26, 96)
sequential_6 	 (1, 26, 26, 256)
max_pooling2d_4 	 (1, 12, 12, 256)
sequential_7 	 (1, 12, 12, 384)
max_pooling2d_5 	 (1, 5, 5, 384)
dropout_1 	 (1, 5, 5, 384)
sequential_8 	 (1, 5, 5, 10)
global_average_pooling2d_1 	 (1, 10)
flatten_1 	 (1, 10)
Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_5 (Sequential)    multiple                  30336     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 multiple                  0         
_________________________________________________________________
sequential_6 (Sequential)    multiple                  746240    
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 multiple                  0         
_________________________________________________________________
sequenti

In [17]:
class DataLoader():
    def __init__(self):
        fashion_mnist = tf.keras.datasets.fashion_mnist
        (self.X_train, self.y_train), (self.X_test, self.y_test) = fashion_mnist.load_data()
        self.X_train = np.expand_dims(self.X_train.astype(np.float32)/255.0,axis=-1)
        self.X_test = np.expand_dims(self.X_test.astype(np.float32)/255.0,axis=-1)
        self.y_train = self.y_train.astype(np.int32)
        self.y_test = self.y_test.astype(np.int32)
        self.num_train, self.num_test = self.X_train.shape[0], self.X_test.shape[0]

    def get_batch_train(self, batch_size):
        index = np.random.randint(0, self.num_train, batch_size)
        #need to resize images to (224,224)
        resized_images = tf.image.resize_with_pad(self.X_train[index],224,224,)
        return resized_images.numpy(), self.y_train[index]

    def get_batch_test(self, batch_size):
        index = np.random.randint(0, self.num_test, batch_size)
        #need to resize images to (224,224)
        resized_images = tf.image.resize_with_pad(self.X_test[index],224,224,)
        return resized_images.numpy(), self.y_test[index]

batch_size = 128
dataLoader = DataLoader()
x_batch, y_batch = dataLoader.get_batch_train(batch_size)
print("x_batch shape:",x_batch.shape,"y_batch shape:", y_batch.shape)


x_batch shape: (128, 224, 224, 1) y_batch shape: (128,)


In [18]:
net.compile(
    optimizer=keras.optimizers.Adam(1e-7), 
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy']
)

epochs = 5
num_iter = dataLoader.num_train // batch_size
for ep in range(epochs):
    for n in range(num_iter):
        X_batch, y_batch = dataLoader.get_batch_train(batch_size)
        net.fit(X_batch, y_batch)
        if n % 20 == 0:
            net.save_weights('nin_weights.h5')
            


Train on 128 samples
128/128 [==============================] - 3s 23ms/sample - loss: 3.1537 - accuracy: 0.0859
Train on 128 samples
128/128 [==============================] - 0s 2ms/sample - loss: 3.2376 - accuracy: 0.0781
Train on 128 samples
128/128 [==============================] - 0s 2ms/sample - loss: 3.2443 - accuracy: 0.1250
Train on 128 samples
128/128 [==============================] - 0s 2ms/sample - loss: 3.2666 - accuracy: 0.0469
Train on 128 samples
128/128 [==============================] - 0s 2ms/sample - loss: 3.3268 - accuracy: 0.0781
Train on 128 samples
128/128 [==============================] - 0s 2ms/sample - loss: 3.0405 - accuracy: 0.1328
Train on 128 samples
128/128 [==============================] - 0s 2ms/sample - loss: 3.3320 - accuracy: 0.0703
Train on 128 samples
128/128 [==============================] - 0s 2ms/sample - loss: 3.3004 - accuracy: 0.0703
Train on 128 samples
128/128 [==============================] - 0s 2ms/sample - loss: 3.2905 - accuracy

128/128 [==============================] - 0s 2ms/sample - loss: 3.1611 - accuracy: 0.0625
Train on 128 samples
128/128 [==============================] - 0s 2ms/sample - loss: 2.8166 - accuracy: 0.0938
Train on 128 samples
128/128 [==============================] - 0s 2ms/sample - loss: 3.2587 - accuracy: 0.0547
Train on 128 samples
128/128 [==============================] - 0s 2ms/sample - loss: 3.0444 - accuracy: 0.1406
Train on 128 samples
128/128 [==============================] - 0s 2ms/sample - loss: 3.1885 - accuracy: 0.0625
Train on 128 samples
128/128 [==============================] - 0s 2ms/sample - loss: 2.8141 - accuracy: 0.1172
Train on 128 samples
128/128 [==============================] - 0s 2ms/sample - loss: 2.8784 - accuracy: 0.0391
Train on 128 samples
128/128 [==============================] - 0s 2ms/sample - loss: 2.9517 - accuracy: 0.0938
Train on 128 samples
128/128 [==============================] - 0s 2ms/sample - loss: 3.0621 - accuracy: 0.0703
Train on 128 

128/128 [==============================] - 0s 2ms/sample - loss: 2.8807 - accuracy: 0.0859
Train on 128 samples
128/128 [==============================] - 0s 2ms/sample - loss: 2.9227 - accuracy: 0.0703
Train on 128 samples
128/128 [==============================] - 0s 2ms/sample - loss: 2.9305 - accuracy: 0.0703
Train on 128 samples
128/128 [==============================] - 0s 2ms/sample - loss: 2.7467 - accuracy: 0.0938
Train on 128 samples
128/128 [==============================] - 0s 2ms/sample - loss: 2.8214 - accuracy: 0.0859
Train on 128 samples
128/128 [==============================] - 0s 2ms/sample - loss: 2.9348 - accuracy: 0.1172
Train on 128 samples
128/128 [==============================] - 0s 2ms/sample - loss: 3.1994 - accuracy: 0.0469
Train on 128 samples
128/128 [==============================] - 0s 2ms/sample - loss: 3.0100 - accuracy: 0.0312
Train on 128 samples
128/128 [==============================] - 0s 2ms/sample - loss: 2.8801 - accuracy: 0.0859
Train on 128 

128/128 [==============================] - 0s 2ms/sample - loss: 2.7761 - accuracy: 0.1094
Train on 128 samples
128/128 [==============================] - 0s 2ms/sample - loss: 2.7204 - accuracy: 0.0781
Train on 128 samples
128/128 [==============================] - 0s 2ms/sample - loss: 2.6325 - accuracy: 0.0781
Train on 128 samples
128/128 [==============================] - 0s 2ms/sample - loss: 2.9125 - accuracy: 0.0703
Train on 128 samples
128/128 [==============================] - 0s 2ms/sample - loss: 2.6781 - accuracy: 0.1016
Train on 128 samples
128/128 [==============================] - 0s 2ms/sample - loss: 2.6199 - accuracy: 0.1016
Train on 128 samples
128/128 [==============================] - 0s 2ms/sample - loss: 2.7976 - accuracy: 0.0547
Train on 128 samples
128/128 [==============================] - 0s 2ms/sample - loss: 2.8376 - accuracy: 0.0703
Train on 128 samples
128/128 [==============================] - 0s 2ms/sample - loss: 2.8030 - accuracy: 0.1328
Train on 128 

128/128 [==============================] - 0s 2ms/sample - loss: 2.6118 - accuracy: 0.1016
Train on 128 samples
128/128 [==============================] - 0s 2ms/sample - loss: 2.7271 - accuracy: 0.1016
Train on 128 samples
128/128 [==============================] - 0s 2ms/sample - loss: 2.7116 - accuracy: 0.0938
Train on 128 samples
128/128 [==============================] - 0s 2ms/sample - loss: 2.7563 - accuracy: 0.0781
Train on 128 samples
128/128 [==============================] - 0s 2ms/sample - loss: 2.7200 - accuracy: 0.0781
Train on 128 samples
128/128 [==============================] - 0s 2ms/sample - loss: 2.5891 - accuracy: 0.1094
Train on 128 samples
128/128 [==============================] - 0s 2ms/sample - loss: 2.6462 - accuracy: 0.1016
Train on 128 samples
128/128 [==============================] - 0s 2ms/sample - loss: 2.8051 - accuracy: 0.0547
Train on 128 samples
128/128 [==============================] - 0s 2ms/sample - loss: 2.8094 - accuracy: 0.0938
Train on 128 

...

128/128 [==============================] - 0s 2ms/sample - loss: 2.2127 - accuracy: 0.1250
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 2.1682 - accuracy: 0.1250
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 2.2340 - accuracy: 0.1172
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 2.1949 - accuracy: 0.1484
Train on 128 samples
128/128 [==============================] - 0s 2ms/sample - loss: 2.1863 - accuracy: 0.1172
Train on 128 samples
128/128 [==============================] - 0s 2ms/sample - loss: 2.1791 - accuracy: 0.1953
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 2.2490 - accuracy: 0.0781
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 2.1790 - accuracy: 0.1328
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 2.2528 - accuracy: 0.1172
Train on 128 

128/128 [==============================] - 0s 2ms/sample - loss: 2.1889 - accuracy: 0.1016
Train on 128 samples
128/128 [==============================] - 0s 2ms/sample - loss: 2.2147 - accuracy: 0.1250
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 2.2061 - accuracy: 0.1328
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 2.2421 - accuracy: 0.0859
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 2.1913 - accuracy: 0.1328
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 2.1819 - accuracy: 0.1953
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 2.1636 - accuracy: 0.1328
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 2.1816 - accuracy: 0.1406
Train on 128 samples
128/128 [==============================] - 0s 2ms/sample - loss: 2.2095 - accuracy: 0.1562
Train on 128 

128/128 [==============================] - 0s 3ms/sample - loss: 2.1800 - accuracy: 0.1562
Train on 128 samples
128/128 [==============================] - 0s 2ms/sample - loss: 2.1939 - accuracy: 0.1484
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 2.1865 - accuracy: 0.1328
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 2.1860 - accuracy: 0.1719
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 2.1818 - accuracy: 0.1641
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 2.2132 - accuracy: 0.1406
Train on 128 samples
128/128 [==============================] - 0s 2ms/sample - loss: 2.2039 - accuracy: 0.1406
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 2.1528 - accuracy: 0.1484
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 2.1866 - accuracy: 0.1328
Train on 128 

128/128 [==============================] - 0s 3ms/sample - loss: 2.1835 - accuracy: 0.1094
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 2.1679 - accuracy: 0.1484
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 2.1217 - accuracy: 0.1719
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 2.2172 - accuracy: 0.0859
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 2.1686 - accuracy: 0.1250
Train on 128 samples
128/128 [==============================] - 0s 2ms/sample - loss: 2.1543 - accuracy: 0.1875
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 2.2161 - accuracy: 0.1250
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 2.1673 - accuracy: 0.1797
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 2.1932 - accuracy: 0.1484
Train on 128 

128/128 [==============================] - 0s 2ms/sample - loss: 2.1268 - accuracy: 0.1719
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 2.1650 - accuracy: 0.1484
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 2.1737 - accuracy: 0.1641
Train on 128 samples
128/128 [==============================] - 0s 3ms/sample - loss: 2.1671 - accuracy: 0.1484


In [19]:
net.load_weights('nin_weights.h5')
X_test, y_test = dataLoader.get_batch_test(2000)
net.evaluate(X_test, y_test, verbose=2)

2000/2000 - 2s - loss: 2.1644 - accuracy: 0.3395


[2.16444229221344, 0.3395]